# GraphRAG Patterns with Neo4j

This notebook demonstrates how to use Neo4j's GraphRAG library to implement retrieval-augmented generation with graph context.

In [ ]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings
from neo4j_graphrag.indexes import create_vector_index
from neo4j_graphrag.retrievers import VectorRetriever, VectorCypherRetriever
from neo4j_graphrag import GraphRAG, RagTemplate

# Load environment variables
load_dotenv()

# Initialize Neo4j connection
URI = os.getenv('NEO4J_URI')
AUTH = (os.getenv('NEO4J_USERNAME'), os.getenv('NEO4J_PASSWORD'))
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Create Neo4j driver
driver = GraphDatabase.driver(URI, auth=AUTH)

# Initialize LLM and embeddings
llm = OpenAILLM(
    model_name="gpt-4",
    model_params={
        "temperature": 0  # Lower temperature for more deterministic results
    }
)

embedder = OpenAIEmbeddings()

## Create Vector Index

First, let's create a vector index for our documents:

In [ ]:
# Create vector index for document embeddings
create_vector_index(
    driver,
    name="document_embeddings",
    label="Document",
    embedding_property="embedding",
    dimensions=1536,
    similarity_fn="cosine"
)

## Pattern 1: Simple Vector Retrieval

Let's start with basic vector similarity search:

In [ ]:
# Initialize vector retriever
vector_retriever = VectorRetriever(
    driver,
    index_name="document_embeddings",
    embedder=embedder,
    return_properties=["text", "title"]
)

# Create RAG template
rag_template = RagTemplate(
    template="""Answer the Question using the following Context. Only respond with information mentioned in the Context.

# Question:
{query_text}

# Context:
{context}

# Answer:
""",
    expected_inputs=['query_text', 'context']
)

# Initialize GraphRAG with vector retriever
vector_rag = GraphRAG(llm=llm, retriever=vector_retriever, prompt_template=rag_template)

# Example query
question = "What are the key features of our laptop products?"
result = vector_rag.search(question, retriever_config={'top_k': 3})
print(f"Answer: {result.answer}")

## Pattern 2: Graph-Enhanced Retrieval

Now let's use graph relationships to enhance our search:

In [ ]:
# Initialize graph-aware retriever
graph_retriever = VectorCypherRetriever(
    driver,
    index_name="document_embeddings",
    embedder=embedder,
    retrieval_query="""
    // Start with similar documents
    WITH node AS doc
    
    // Get product information
    MATCH (doc)-[:ABOUT]->(p:Product)
    
    // Get related support cases and manuals
    OPTIONAL MATCH (p)<-[:ABOUT]-(support:Document)
    WHERE support.type = 'support_case'
    OPTIONAL MATCH (p)-[:HAS_MANUAL]->(manual:Document)
    
    // Return combined context
    WITH collect(DISTINCT doc.text) + 
         collect(DISTINCT support.text) +
         collect(DISTINCT manual.text) as texts
    RETURN apoc.text.join(texts, '\n') as info
    """
)

# Initialize GraphRAG with graph retriever
graph_rag = GraphRAG(llm=llm, retriever=graph_retriever, prompt_template=rag_template)

# Example query using graph context
question = "What are common issues with the Laptop Pro model and their solutions?"
result = graph_rag.search(question, retriever_config={'top_k': 3})
print(f"Answer: {result.answer}")

## Pattern 3: Multi-Hop Knowledge Retrieval

Let's use graph traversal to gather related information:

In [ ]:
# Initialize multi-hop retriever
multihop_retriever = VectorCypherRetriever(
    driver,
    index_name="document_embeddings",
    embedder=embedder,
    retrieval_query="""
    // Start with similar documents
    WITH node AS doc
    
    // Get product and category information
    MATCH (doc)-[:ABOUT]->(p:Product)-[:IN_CATEGORY]->(cat:Category)
    
    // Get related products in same category
    MATCH (cat)<-[:IN_CATEGORY]-(related:Product)
    WHERE related <> p
    
    // Get support cases for all related products
    MATCH (related)<-[:ABOUT]-(support:Document)
    WHERE support.type = 'support_case'
    
    // Return combined context with relationship information
    WITH collect(DISTINCT doc.text) + 
         collect(DISTINCT support.text) as texts,
         p.name as product,
         cat.name as category,
         collect(DISTINCT related.name) as related_products
    
    RETURN 
        apoc.text.join(texts, '\n') +
        '\nProduct: ' + product +
        '\nCategory: ' + category +
        '\nRelated Products: ' + apoc.text.join(related_products, ', ') as info
    """
)

# Initialize GraphRAG with multi-hop retriever
multihop_rag = GraphRAG(llm=llm, retriever=multihop_retriever, prompt_template=rag_template)

# Example query using multi-hop traversal
question = "What are common issues across our laptop product line?"
result = multihop_rag.search(question, retriever_config={'top_k': 3})
print(f"Answer: {result.answer}")

## Key GraphRAG Features Used

1. **Vector Indexing**: Using Neo4j's vector search capabilities
2. **Vector Retrieval**: Basic similarity search using embeddings
3. **Graph-Enhanced Retrieval**: Combining vector search with graph traversal
4. **Cypher Integration**: Custom graph queries for context gathering
5. **Multi-Hop Knowledge**: Traversing relationships to gather related information